# Introduction

This program performs analysis and visualization of openLCA results from an LCIA calculation, starting with an excel file export. This program assumes that you've already compiled an excel file of the unit processes exported from your openLCA database, with ISIC categories assigned to each process, using the 